# Import Stuff

In [1]:
import os
import time
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import onnx
import onnxruntime

from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt

from matching import matching
from evaluation.metrics import createPR, recallAt100precision, recallAtK
from datasets.load_dataset import GardensPointDataset, SFUDataset, StLuciaDataset

# conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
# conda install onnx onnxruntime
# pip install torch torchvision numpy opencv-python seaborn matplotlib scikit-learn pillow onnxscript

c:\Users\Irfan Q\miniconda3\envs\bruh\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


# Load Datasets

In [2]:
imgs_db, imgs_q, GThard, GTsoft = SFUDataset().load()
# imgs_db, imgs_q, GThard, GTsoft = GardensPointDataset().load()
# imgs_db, imgs_q, GThard, GTsoft = StLuciaDataset().load()

===== Load dataset SFU dry--jan


# Constants

In [3]:
WEIGHTS_FILE = "calc.caffemodel.pt"
ITERATIONS = 100

# Preprocess Images

In [4]:
class ConvertToYUVandEqualizeHist:
    def __call__(self, img):
        img_yuv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        img_rgb = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
        return Image.fromarray(img_rgb)

preprocess = transforms.Compose(
    [
        ConvertToYUVandEqualizeHist(),
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((120, 160), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
    ]
)

In [5]:
# Preprocess query images

query_matrix = []

for image in imgs_db:
    preprocessed_image = preprocess(image)
    query_matrix.append(preprocessed_image)

query_matrix = np.stack(query_matrix, axis=0)

# Convert numpy array to a tensor
query_tensor = torch.from_numpy(query_matrix)

print(query_tensor.shape)
print(query_tensor)

torch.Size([385, 1, 120, 160])
tensor([[[[0.3098, 0.4431, 0.6235,  ..., 0.0314, 0.0196, 0.0196],
          [0.1882, 0.4471, 0.7020,  ..., 0.0471, 0.0353, 0.0353],
          [0.1412, 0.4392, 0.6510,  ..., 0.0431, 0.0314, 0.0353],
          ...,
          [0.7882, 0.8157, 0.8314,  ..., 0.1529, 0.1176, 0.0824],
          [0.7961, 0.8118, 0.8353,  ..., 0.1333, 0.0902, 0.0627],
          [0.7843, 0.8000, 0.8196,  ..., 0.0980, 0.0588, 0.0431]]],


        [[[0.6275, 0.6039, 0.5804,  ..., 0.8118, 0.8000, 0.7569],
          [0.5216, 0.5333, 0.5176,  ..., 0.8157, 0.7922, 0.7333],
          [0.3922, 0.4863, 0.4706,  ..., 0.8078, 0.7765, 0.7294],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.2667, 0.1843, 0.1373],
          [1.0000, 1.0000, 1.0000,  ..., 0.2392, 0.1373, 0.1098],
          [1.0000, 1.0000, 1.0000,  ..., 0.1490, 0.0980, 0.0588]]],


        [[[0.0902, 0.2039, 0.3294,  ..., 0.8392, 0.8980, 0.9451],
          [0.1529, 0.2510, 0.3490,  ..., 0.8627, 0.8706, 0.9412],
       

In [6]:
# Preprocess map images

map_matrix = []

for image in imgs_q:
    preprocessed_image = preprocess(image)
    map_matrix.append(preprocessed_image)

map_matrix = np.stack(map_matrix, axis=0)

# Convert numpy array to a tensor
map_tensor = torch.from_numpy(map_matrix)

print(map_tensor.shape)
print(map_tensor)

torch.Size([385, 1, 120, 160])
tensor([[[[0.5725, 0.6667, 0.6784,  ..., 0.3098, 0.1569, 0.0941],
          [0.2157, 0.5686, 0.7490,  ..., 0.2588, 0.1569, 0.0941],
          [0.0667, 0.1804, 0.6667,  ..., 0.2235, 0.1569, 0.1059],
          ...,
          [0.2980, 0.3529, 0.4314,  ..., 0.8196, 0.7961, 0.7725],
          [0.1725, 0.2863, 0.4196,  ..., 0.8196, 0.7882, 0.7490],
          [0.1725, 0.2745, 0.3765,  ..., 0.8118, 0.7569, 0.6824]]],


        [[[0.8627, 0.9451, 0.9373,  ..., 0.9647, 0.8980, 0.8275],
          [0.8980, 0.9059, 0.9451,  ..., 0.9804, 0.9333, 0.8824],
          [0.4314, 0.6039, 0.9176,  ..., 0.9804, 0.9529, 0.9137],
          ...,
          [0.3137, 0.3922, 0.4784,  ..., 0.5451, 0.5961, 0.6118],
          [0.2667, 0.3294, 0.3804,  ..., 0.5137, 0.5255, 0.5529],
          [0.1922, 0.2588, 0.3098,  ..., 0.5020, 0.4941, 0.5059]]],


        [[[0.0039, 0.0000, 0.0039,  ..., 0.9765, 0.9373, 0.7765],
          [0.0039, 0.0039, 0.0078,  ..., 0.9686, 0.9569, 0.8667],
       

# Model Definition

In [16]:
class CalcModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.input_dim = (1, 120, 160)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(5, 5), stride=2, padding=4)
        self.relu1 = nn.ReLU(inplace=False)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(4, 4), stride=1, padding=2)
        self.relu2 = nn.ReLU(inplace=False)
        self.conv3 = nn.Conv2d(128, 4, kernel_size=(3, 3), stride=1, padding=0)
        self.relu3 = nn.ReLU(inplace=False)
        self.pool = nn.MaxPool2d(kernel_size=(3, 3), stride=2)
        self.lrn1 = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        self.lrn2 = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)


    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool(x)
        x = self.lrn1(x)

        x = self.relu2(self.conv2(x))
        x = self.pool(x)
        x = self.lrn2(x)

        x = self.relu3(self.conv3(x))
        x = torch.flatten(x, 1)
        return x

### Normal Model

In [17]:
calc = CalcModel()

# Load the model weights
state_dict = torch.load(WEIGHTS_FILE)
my_new_state_dict = {}
my_layers = list(calc.state_dict().keys())
for layer in my_layers:
    my_new_state_dict[layer] = state_dict[layer]
calc.load_state_dict(my_new_state_dict)

print(calc)

CalcModel(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(4, 4))
  (relu1): ReLU()
  (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (conv3): Conv2d(128, 4, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (pool): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
)


### Quantized Model - Pytorch doesn't work?

In [ ]:
# quantized_model = torch.quantization.quantize_dynamic(
#     calc,
#     {nn.Conv2d, nn.MaxPool2d, nn.LocalResponseNorm},
#     dtype=torch.qint8
# )

# print(quantized_model)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.quantization
import copy

quantized_model = CalcModel()
quantized_model.eval()

backend = "x86"
# for raspberry pi should be set to "qnnpack"
# try onnx quantization

## EAGER MODE

torch.quantization.fuse_modules(quantized_model, [['conv1', 'relu1'], ['conv2', 'relu2'], ['conv3', 'relu3']], inplace=True)

quantized_model = nn.Sequential(torch.quantization.QuantStub(), 
                  quantized_model, 
                  torch.quantization.DeQuantStub())

# Prepare
quantized_model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.quantization.prepare(quantized_model, inplace=True)

# Calibrate
with torch.inference_mode():
  for i in query_tensor[:100]: # idk if this is the right way to do this
    quantized_model(i)

# Convert
torch.quantization.convert(quantized_model, inplace=True)

# Check
print(quantized_model) # 1 byte instead of 4 bytes for FP32

c:\Users\Irfan Q\miniconda3\envs\bruh\Lib\site-packages\torch\ao\quantization\observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Sequential(
  (0): Quantize(scale=tensor([0.0079]), zero_point=tensor([0]), dtype=torch.quint8)
  (1): CalcModel(
    (conv1): QuantizedConvReLU2d(1, 64, kernel_size=(5, 5), stride=(2, 2), scale=0.009876943193376064, zero_point=0, padding=(4, 4))
    (relu1): Identity()
    (conv2): QuantizedConvReLU2d(64, 128, kernel_size=(4, 4), stride=(1, 1), scale=0.00443233922123909, zero_point=0, padding=(2, 2))
    (relu2): Identity()
    (conv3): QuantizedConvReLU2d(128, 4, kernel_size=(3, 3), stride=(1, 1), scale=0.0008810117724351585, zero_point=0)
    (relu3): Identity()
    (pool): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
    (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
    (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  )
  (2): DeQuantize()
)


In [ ]:
# ## FX GRAPH
# from torch.quantization import quantize_fx

# m = copy.deepcopy(calc)
# m.eval()

# qconfig_dict = {"": torch.quantization.get_default_qconfig(backend)}

# # Prepare
# example_inputs = (torch.rand(1, 1, 120, 160))

# model_prepared = quantize_fx.prepare_fx(m, qconfig_dict, example_inputs)

# # Calibrate
# with torch.inference_mode():
#   for i in query_tensor[:100]:
#     model_prepared(i)

# # Quantize
# model_quantized = quantize_fx.convert_fx(model_prepared)

### ONNX Model

In [ ]:
example_input = torch.randn(1, 1, 120, 160)

dynamic_axes = {"input": {0: "batch_size"}, "output": {0: "batch_size"}}

# Export the model
torch.onnx.export(
    calc,  # model
    example_input,  # example input
    "calc_model.onnx",  # output file name
    input_names=["input"],  # input names
    output_names=["output"],  # output names
    dynamic_axes=dynamic_axes,  # dynamic axes
)

### CUDA Model

In [ ]:
torch.cuda.is_available()

# Instantiate the Model
calc_cuda = CalcModel().half().cuda()

# Load the model weights
state_dict = torch.load(WEIGHTS_FILE)
my_new_state_dict = {}
my_layers = list(calc_cuda.state_dict().keys())
for layer in my_layers:
    my_new_state_dict[layer] = state_dict[layer]
calc_cuda.load_state_dict(my_new_state_dict)

# Run Models

### Normal Model

In [20]:
query_tensor = torch.from_numpy(query_matrix)
map_tensor = torch.from_numpy(map_matrix)

print(query_tensor.shape)
print(map_tensor.shape)

query_tensor = query_tensor.view(-1, 1, 120, 160)
map_tensor = map_tensor.view(-1, 1, 120, 160)

# Pass the tensors through the model

query_features = calc(query_tensor)
map_features = calc(map_tensor)

print(query_features.shape)
print(map_features.shape)

torch.Size([385, 1, 120, 160])
torch.Size([385, 1, 120, 160])
torch.Size([385, 936])
torch.Size([385, 936])


### Quantized Model

In [21]:
quan_query_features = quantized_model(query_tensor)
quan_map_features = quantized_model(map_tensor)

print(quan_query_features.shape)
print(quan_map_features.shape)

NotImplementedError: Could not run 'aten::mul.out' with arguments from the 'QuantizedCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::mul.out' is only available for these backends: [CPU, CUDA, Meta, MkldnnCPU, SparseCPU, SparseCUDA, SparseCsrCPU, SparseCsrCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode

In [ ]:
# Convert pytorch tensors to numpy arrays
# query_features_np = query_features.detach().numpy()
# map_features_np = map_features.detach().numpy()

# similarity_matrix = cosine_similarity(query_features_np, map_features_np)

# print(similarity_matrix.shape)

In [ ]:
# plt.figure()
# sns.heatmap(similarity_matrix, cmap='viridis')
# plt.title('Similarity matrix')
# plt.axis('off')
# plt.show()

In [ ]:
# # best matching per query in S for single-best-match VPR
# M1 = matching.best_match_per_query(similarity_matrix)

# # find matches with S>=thresh using an auto-tuned threshold for multi-match VPR
# M2 = matching.thresholding(similarity_matrix, 'auto')

# TP = np.argwhere(M2 & GThard)  # true positives
# FP = np.argwhere(M2 & ~GTsoft)  # false positives

# # show M's
# fig = plt.figure()
# ax1 = fig.add_subplot(121)
# ax1.imshow(M1)
# ax1.axis('off')
# ax1.set_title('Best match per query')
# ax2 = fig.add_subplot(122)
# ax2.imshow(M2)
# ax2.axis('off')
# ax2.set_title('Thresholding S>=thresh')

### ONNX Model

In [ ]:
# Check if model is a valid ONNX model
onnx_model = onnx.load("calc_model.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
# Convert the tensors to numpy arrays
query_numpy = query_tensor.detach().cpu().numpy()
map_numpy = map_tensor.detach().cpu().numpy()

ort_session = onnxruntime.InferenceSession("calc_model.onnx")

In [ ]:
# Get the input name from the model
input_name = ort_session.get_inputs()[0].name

# Ensure the inputs are numpy arrays
query_numpy = np.array(query_numpy)
map_numpy = np.array(map_numpy)

# Create the input dictionary
ort_query_input = {input_name: query_numpy}

# Run the model
ort_query_output = ort_session.run(None, ort_query_input)

# Convert the output to a numpy array and print its shape
ort_query_output = np.array(ort_query_output)
print(ort_query_output.shape)

# Create the input dictionary
ort_map_input = {input_name: map_numpy}

# Run the model
ort_map_output = ort_session.run(None, ort_map_input)

# Convert the output to a numpy array and print it
ort_map_output = np.array(ort_map_output)
print(ort_map_output.shape)

In [ ]:
ort_query_output = np.squeeze(ort_query_output)
print(ort_query_output.shape)
ort_map_output = np.squeeze(ort_map_output)
print(ort_map_output.shape)

### CUDA Model

In [ ]:
# Pass the tensors through the model

query_features = calc_cuda(query_tensor.half().cuda())
map_features = calc_cuda(map_tensor.half().cuda())

query_features_cpu = query_features.detach().cpu()
map_features_cpu = map_features.detach().cpu()

print(query_features_cpu.shape)
print(map_features_cpu.shape)

# Average Time

### Normal Model

In [ ]:
# Average time for query tensor

start_time = time.time()

for _ in range(ITERATIONS):
    output = calc(query_tensor)

end_time = time.time()

query_duration_calc = end_time - start_time
query_avg_duration_calc = query_duration_calc / ITERATIONS

print(f"Time taken: {query_duration_calc} seconds")
print(f"Average time taken: {query_avg_duration_calc} seconds")

# Average time for map tensor

start_time = time.time()

for _ in range(ITERATIONS):
    with torch.no_grad():
        output = calc(map_tensor)

end_time = time.time()

map_duration_calc = end_time - start_time
map_avg_duration_calc = map_duration_calc / ITERATIONS

print(f"Time taken: {map_duration_calc} seconds")
print(f"Average time taken: {map_avg_duration_calc} seconds")

### Quantized Model

In [ ]:
# Average time for query tensor

start_time = time.time()

for _ in range(ITERATIONS):
    output = quantized_model(query_tensor)

end_time = time.time()

query_duration_calc = end_time - start_time
query_avg_duration_calc = query_duration_calc / ITERATIONS

print(f"Time taken: {query_duration_calc} seconds")
print(f"Average time taken: {query_avg_duration_calc} seconds")

# Average time for map tensor

start_time = time.time()

for _ in range(ITERATIONS):
    with torch.no_grad():
        output = quantized_model(map_tensor)

end_time = time.time()

map_duration_calc = end_time - start_time
map_avg_duration_calc = map_duration_calc / ITERATIONS

print(f"Time taken: {map_duration_calc} seconds")
print(f"Average time taken: {map_avg_duration_calc} seconds")

### ONNX Model

In [ ]:
# Average time for query tensor

input_name = ort_session.get_inputs()[0].name
ort_inputs = {input_name: query_numpy}

start_time = time.time()

for _ in range(ITERATIONS):
    ort_outs = ort_session.run(None, ort_inputs)

end_time = time.time()

query_duration_onnx = end_time - start_time
query_avg_duration_onnx = query_duration_onnx / ITERATIONS

print(f"Time taken: {query_duration_onnx} seconds")
print(f"Average time taken: {query_avg_duration_onnx} seconds")

# Average time for map tensor

input_name = ort_session.get_inputs()[0].name
ort_inputs = {input_name: map_numpy}

start_time = time.time()

for _ in range(ITERATIONS):
    ort_outs = ort_session.run(None, ort_inputs)

end_time = time.time()

map_duration_onnx = end_time - start_time
map_avg_duration_onnx = map_duration_onnx / ITERATIONS

print(f"Time taken: {map_duration_onnx} seconds")
print(f"Average time taken: {map_avg_duration_onnx} seconds")

### CUDA Model

In [ ]:
start_time = time.time()

for _ in range(ITERATIONS):
    output = calc_cuda(query_tensor.half().cuda())

end_time = time.time()

query_duration_cuda = end_time - start_time
query_avg_duration_cuda = query_duration_cuda / ITERATIONS

print(f"Time taken: {query_duration_cuda} seconds")
print(f"Average time taken: {query_avg_duration_cuda} seconds")

# Average time for map tensor

start_time = time.time()

for _ in range(ITERATIONS):
    output = calc_cuda(map_tensor.half().cuda())

end_time = time.time()

map_duration_cuda = end_time - start_time
map_avg_duration_cuda = map_duration_cuda / ITERATIONS

print(f"Time taken: {map_duration_cuda} seconds")
print(f"Average time taken: {map_avg_duration_cuda} seconds")

# Evaluation

In [ ]:
import matplotlib.pyplot as plt

# Variables
models = ['Normal', 'ONNX', 'CUDA']
durations = [query_duration_calc, map_duration_onnx, map_duration_cuda]
avg_durations = [query_avg_duration_calc, map_avg_duration_onnx, map_avg_duration_cuda]

# Plot duration bar chart
plt.figure(figsize=(5, 3))
plt.bar(models, durations)
plt.title('Duration')
plt.ylabel('Duration')
plt.show()

# Plot average duration bar chart
plt.figure(figsize=(5, 3))
plt.bar(models, avg_durations)
plt.title('Average Duration')
plt.ylabel('Average Duration')
plt.show()

In [ ]:
# precision-recall curve
P, R = createPR(similarity_matrix, GThard, GTsoft, matching='multi', n_thresh=100)
plt.figure()
plt.plot(R, P)
plt.xlim(0, 1), plt.ylim(0, 1.01)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Result on GardensPoint day_right--night_right')
plt.grid('on')
plt.draw()
plt.show()